## Facebook Data Case Study

### By Prakhar Gupta

Please walk through the problem statement file and the 'Data' tab in the "Sample Data" workbook for reference.

For this case study, the dataset "Sample Data" has ben already preprocessed, where the empty cells in the "Post creation date" and the "Page ID" columns were filled along with some formatting of the date (Year-month-day)

Let's import the libraries 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel("Sample Data.xlsx")

In [3]:
df.head()

,Post creation date,Page ID,Post ID,Post creation time of day,Video length (s),Post reach,Post reactions,Post comments,Post shares,Video views,Completed video views,Avg. video view time (s)
0,2021-06-01,FB003,FB003_743065,10:06:00,78.016,64539,1540,60,296,30983,3828,13.77
1,2021-06-01,FB003,FB003_855641,13:12:01,NaN,94086,2020,313,336,0,0,0.00
2,2021-06-01,FB003,FB003_859546,15:21:19,171.051,65406,1058,54,380,30116,725,14.23
3,2021-06-01,FB003,FB003_177590,16:01:00,296.128,271591,4521,425,2213,114324,8384,22.21
4,2021-06-01,FB003,FB003_173788,17:06:01,NaN,44813,772,88,157,0,0,0.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Post creation date         1539 non-null   datetime64[ns]
 1   Page ID                    1539 non-null   object        
 2   Post ID                    1539 non-null   object        
 3   Post creation time of day  1539 non-null   object        
 4   Video length (s)           735 non-null    float64       
 5   Post reach                 1539 non-null   int64         
 6   Post reactions             1539 non-null   int64         
 7   Post comments              1539 non-null   int64         
 8   Post shares                1539 non-null   int64         
 9   Video views                1539 non-null   int64         
 10  Completed video views      1539 non-null   int64         
 11  Avg. video view time (s)   1539 non-null   float64       
dtypes: dat

### Solution to Question 2: Returning most viewed posts for each page

First, we will make a new field "Cummulative Watch Time" by multiplying the views and average view time fields. 

In [5]:
df['Cummulative Watch Time'] = df['Avg. video view time (s)'] * df['Video views']

Identifying unique page IDs in the dataframe 

In [6]:
df['Page ID'].unique()

array(['FB003', 'FB001', 'FB002', 'FB004'], dtype=object)

#### Defining the "most_watched_post":

The fuction takes in the Page ID and will return the Post ID, Creation Date, Time, Length of the video and Cummulative Watch Time in a dataframe format 

In [9]:
def most_watched_post(pages = df['Page ID'].unique()):
    
    df_list = []
    
    for pageid in pages:
    
        # Filtering the dataset W.R.T Page ID
        d = df[df["Page ID"] == pageid]
    
        # Further filtering W.R.T max Cummulative Watch Time
        max_df =  d[d["Cummulative Watch Time"] == d["Cummulative Watch Time"].max()]
    
        # Appending the Dataframes to the list
        df_list.append(max_df[['Page ID','Post ID','Post creation date','Post creation time of day','Video length (s)','Cummulative Watch Time']])
    
        # Concatenating all the DataFrames into one
        final_df = pd.concat([df for df in df_list], axis=0, ignore_index=True)
        
    return final_df.sort_values('Page ID')

In [10]:
most_watched_post()

,Page ID,Post ID,Post creation date,Post creation time of day,Video length (s),Cummulative Watch Time
1,FB001,FB001_579376,2021-06-18,19:11:00,180.608,1.619899e+07
2,FB002,FB002_533099,2021-06-11,08:01:18,135.765,1.681556e+08
0,FB003,FB003_749362,2021-06-23,15:22:37,176.448,6.622553e+07
3,FB004,FB004_203587,2021-07-23,20:13:01,162.000,1.171663e+07


In [11]:
df['Page ID'].unique()[0]

'FB003'

#### Most Shared Post: 

Furthermore, we can repeat the aforementioned step for most shared post as well.

In [12]:
def most_shared_post(pages = df['Page ID'].unique()):

    df_list =[]    
    
    for pageid in pages:
        
        # Filtering the dataset W.R.T Page ID
        d = df[df["Page ID"] == pageid]
        
        # Further filtering W.R.T max Cummulative Watch Time
        max_df =  d[d["Post shares"] == d["Post shares"].max()]
    
        # Appending the Dataframes to the list
        df_list.append(max_df[['Page ID','Post ID','Post creation date','Post creation time of day','Post shares','Post comments','Post reach','Post reactions']])
    
        # Concatenating all the DataFrames into one
        final_df = pd.concat([df for df in df_list], axis=0, ignore_index=True)
        
    return final_df.sort_values('Page ID')   

In [13]:
most_shared_post()

,Page ID,Post ID,Post creation date,Post creation time of day,Post shares,Post comments,Post reach,Post reactions
1,FB001,FB001_579376,2021-06-18,19:11:00,6754,757,2174204,24259
2,FB002,FB002_292476,2021-06-25,21:09:00,44530,16534,4367577,61772
0,FB003,FB003_648055,2021-07-13,08:11:00,97974,750,148657,9827
3,FB004,FB004_604343,2021-06-24,20:05:00,1742,864,190014,2273


### Solution to Question 3: Hour of the day with min shares per post

We need to filter the dataframe where the 'Post shares' column is minimum and return the post creation time

In [14]:
min_post_share = df[df['Post shares'] == df['Post shares'].min()]['Post creation time of day']
min_post_share

563    20:12:00
Name: Post creation time of day, dtype: object

In [15]:
type(min_post_share)

pandas.core.series.Series

In [16]:
# Since min_post_share is Series, returning the post creation time will be done in the following manner
min_post_share[min_post_share.index[0]]

datetime.time(20, 12)

In [17]:
min_post_share[min_post_share.index[0]].hour

20

So, the answer to question 3 is 20

#### Supplementary: Creating a function to return the above parameter for each day

While we returned the overall minimum shared post's hour of posting, we can find the same for each day as well.
 
- First, we'll create a dictionary matching each date to its corressponding day no, starting from June 1st. 
- We will then use the 'map' method to make a new column in our original dataframe containing day no(s), by using the above dictonary.
- Finally, we shall write a function to return a separate DataFrame matching the required metric for each day. 

In [18]:
df['Post creation date'][0]

Timestamp('2021-06-01 00:00:00')

In [19]:
# Creating the Dictionary

counter = 0
date_dict = {}

for i in range(len(df)):
    
    if df['Post creation date'][i] not in date_dict:
        date_dict[df['Post creation date'][i]] = counter + 1
        counter += 1

date_dict[df['Post creation date'][1538]]        

61

In [20]:
# Adding the 'Day No.' column

df['Day No.'] = df['Post creation date'].map(date_dict)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Post creation date         1539 non-null   datetime64[ns]
 1   Page ID                    1539 non-null   object        
 2   Post ID                    1539 non-null   object        
 3   Post creation time of day  1539 non-null   object        
 4   Video length (s)           735 non-null    float64       
 5   Post reach                 1539 non-null   int64         
 6   Post reactions             1539 non-null   int64         
 7   Post comments              1539 non-null   int64         
 8   Post shares                1539 non-null   int64         
 9   Video views                1539 non-null   int64         
 10  Completed video views      1539 non-null   int64         
 11  Avg. video view time (s)   1539 non-null   float64       
 12  Cummul

In [21]:
# Writing the Function

def least_shared_hour_per_day(data=df):
    
    #making an empty list which will be filled with the hours
    hours = []

    for day in range(1, 62):
        
        #Filtering the df w.r.t each day
        filtered_df = data[data['Day No.'] == day]
        
        # Identifying the least shared post for the day
        min_shared_post = filtered_df[filtered_df['Post shares'] == filtered_df['Post shares'].min()]['Post creation time of day']
        
        # Returning the hour corressponding to the least shared post
        hour = min_shared_post[min_shared_post.index[0]].hour
        
        # Appending to the list
        hours.append(hour)
        
    return pd.DataFrame({'Day No.':list(range(1, 62)), 'Hour':hours}) 

In [22]:
D = least_shared_hour_per_day().set_index('Day No.')
D.head()

,Hour
Day No.,
1,19
2,17
3,17
4,11
5,13


### Solution to Question 4: Creating a modified dataframe containing a field lbelling the post as Video on Non-Video

A post can be classified as Video or Non-video simply by checking whether it has any value for the metrics related to the videos or not. A non-video post will have a null value for video length field

Here, we are assigning the label of "Non-Video Post" if it has zero views as all the "Video Posts" have some value in this field.  

In [23]:
df['Post Type'] = df['Video views'].apply(lambda x: 'Non-Video Post' if x==0 else 'Video Post')

In [24]:
df.head()

,Post creation date,Page ID,Post ID,Post creation time of day,Video length (s),Post reach,Post reactions,Post comments,Post shares,Video views,Completed video views,Avg. video view time (s),Cummulative Watch Time,Day No.,Post Type
0,2021-06-01,FB003,FB003_743065,10:06:00,78.016,64539,1540,60,296,30983,3828,13.77,426635.91,1,Video Post
1,2021-06-01,FB003,FB003_855641,13:12:01,NaN,94086,2020,313,336,0,0,0.00,0.00,1,Non-Video Post
2,2021-06-01,FB003,FB003_859546,15:21:19,171.051,65406,1058,54,380,30116,725,14.23,428550.68,1,Video Post
3,2021-06-01,FB003,FB003_177590,16:01:00,296.128,271591,4521,425,2213,114324,8384,22.21,2539136.04,1,Video Post
4,2021-06-01,FB003,FB003_173788,17:06:01,NaN,44813,772,88,157,0,0,0.00,0.00,1,Non-Video Post


Since we can only compare the two types of posts in terms of reach, reactions, comments and shares, we don't need other metrics limited to video posts.

In [25]:
new_df = df.drop(['Video length (s)','Video views','Completed video views',
                 'Avg. video view time (s)', 'Cummulative Watch Time'], axis=1)

new_df.head()

,Post creation date,Page ID,Post ID,Post creation time of day,Post reach,Post reactions,Post comments,Post shares,Day No.,Post Type
0,2021-06-01,FB003,FB003_743065,10:06:00,64539,1540,60,296,1,Video Post
1,2021-06-01,FB003,FB003_855641,13:12:01,94086,2020,313,336,1,Non-Video Post
2,2021-06-01,FB003,FB003_859546,15:21:19,65406,1058,54,380,1,Video Post
3,2021-06-01,FB003,FB003_177590,16:01:00,271591,4521,425,2213,1,Video Post
4,2021-06-01,FB003,FB003_173788,17:06:01,44813,772,88,157,1,Non-Video Post


#### Exporting this modified dataframe to an excel file, where a Pivot Chart will be made 

Note: The dataframes have already been exported, hence the codes to export the data will return an error

In [26]:
# new_df.to_excel('Modified Data.xlsx', index=False)

Cross checking whether there are any video posts with zero views 

In [27]:
video_posts = df[df['Video length (s)'].notnull()]
non_video_posts = df[df['Video length (s)'].isnull()]

In [28]:
video_posts[video_posts['Video views']==0]

,Post creation date,Page ID,Post ID,Post creation time of day,Video length (s),Post reach,Post reactions,Post comments,Post shares,Video views,Completed video views,Avg. video view time (s),Cummulative Watch Time,Day No.,Post Type


There are no such video posts!

Let's export these dataframes as well

In [29]:
# video_posts.to_excel('Video Posts.xlsx', index=False)
# non_video_posts.to_excel('Non-Video Posts.xlsx', index=False)

#### That's it for this code! Everything else is done in excel.